In [1]:
# Exploring predict next row by current

!pip install lightgbm

In [1]:
!del /Q utils\__pycache__

In [2]:
import pandas as pd
import numpy as np
from tqdm.notebook import tqdm

In [3]:
from utils.metrics import time_span_metrics
from utils.datasets import GhlKasperskyDataset, TepHarvardDataset, SwatItrustDataset
from utils.custom_plots import plot_stacked

In [4]:
from sklearn.preprocessing import StandardScaler

In [5]:
from lightgbm import LGBMRegressor

# GHL

In [6]:
ds = GhlKasperskyDataset()
ds.shake_not_stir(valid_test_ratio=0.3)

In [7]:
train, _, _ = next(ds.train_generator())
valid, _, _ = next(ds.valid_generator())

In [9]:
train.columns

Index(['input_temp_gc', 'rt_temp_gc', 'ht_temp_gc', 'ct_temp_gc', 'rt_level_m',
       'ht_level_m', 'ct_level_m', 'input_flow_state', 'supply_flow_state',
       'return_flow_state', 'output_flow_state', 'heater_state'],
      dtype='object')

In [10]:
y = train['rt_level_m'].shift(-1).iloc[:-1]
X = train.iloc[:-1]

In [11]:
lr_1 = LGBMRegressor(random_state=31)
lr_1.fit(X, y)
print('Score:', lr_1.score(X, y))
# print(pd.Series(index=train.columns, data=lr_1.coef_))
# print('icpt:', lr_1.intercept_)

Score: 0.9960547905095276


In [12]:
lr_2 = Ridge(random_state=31)
lr_2.fit(X, y)
print('Score:', lr_2.score(X, y))
# print(pd.Series(index=train.columns, data=lr_1.coef_))
# print('icpt:', lr_1.intercept_)

NameError: name 'Ridge' is not defined

In [ ]:
def fit_and_score(lr, df):
    scores = pd.Series(index=df.columns, name='score', dtype='float64')
    X = df.iloc[:-1].copy()  # without last row
    for c in df.columns:
        y = df[c].shift(-1).iloc[:-1]  # predict next value
        lr.fit(X, y)
        scores[c] = lr.score(X, y)
    return scores

In [ ]:
fit_and_score(LinearRegression(), train).median()

In [ ]:
fit_and_score(Lasso(random_state=31), train).median()

In [ ]:
fit_and_score(Ridge(random_state=31), train).median()

In [ ]:
train_ = pd.DataFrame(index=train.index, columns=train.columns, data=StandardScaler().fit_transform(train))
fit_and_score(SGDRegressor(random_state=31), train_)

In [ ]:
# Let's using SGDRegressor, because it have warm_start

In [ ]:
train.info()

# TEP Harvard

In [ ]:
ds = TepHarvardDataset()
ds.shake_not_stir(valid_test_ratio=0.3)

In [ ]:
train, _, _ = next(ds.train_generator())
valid, _, _ = next(ds.valid_generator())

In [ ]:
# add derivatives
def add_derivatives(data: pd.DataFrame) -> pd.DataFrame:
    diff1 = data.diff()
    diff1.columns = data.columns + '_d1'
    diff2 = diff1.diff()
    diff2.columns = data.columns + '_d2'
    return pd.concat([data, diff1, diff2], axis=1).fillna(0)
#     return pd.concat([data, diff1], axis=1).fillna(0)

train = add_derivatives(train)

In [ ]:
train.columns

In [ ]:
fit_and_score(LinearRegression(), train).median()

In [ ]:
fit_and_score(Lasso(random_state=31), train).median()

In [ ]:
fit_and_score(Ridge(random_state=31), train).median()

In [ ]:
train_ = pd.DataFrame(index=train.index, columns=train.columns, data=StandardScaler().fit_transform(train))
fit_and_score(SGDRegressor(random_state=31), train_).median()

In [ ]:
train.info()

# SWaT

In [ ]:
ds = SwatItrustDataset()
ds.shake_not_stir(valid_test_ratio=0.3)

In [ ]:
train, _, _ = next(ds.train_generator())
valid, _, _ = next(ds.valid_generator())

In [ ]:
fit_and_score(LinearRegression(), train).median()

In [ ]:
# fit_and_score(Lasso(random_state=31), train).median()

In [ ]:
# fit_and_score(Ridge(random_state=31), train).median()

In [ ]:
regr = SGDRegressor(random_state=31, 
                    penalty='l2', 
                    alpha=1.0, 
                    l1_ratio=0.15, 
                    max_iter=1000, 
                    tol=0.001, 
                    learning_rate='invscaling', 
                    eta0=0.01, 
                    power_t=0.25, 
                    validation_fraction=0.1, 
                    n_iter_no_change=10, 
                    warm_start=False, 
                    average=False)
train_ = pd.DataFrame(index=train.index, columns=train.columns, data=StandardScaler().fit_transform(train))
fit_and_score(regr, train_)

In [ ]:
train.info()

# Watchman

In [8]:
from utils.watchmen import GbmPredictWatchman

## GHL

In [9]:
ds = GhlKasperskyDataset()
ds.shake_not_stir(random_state=42, valid_test_ratio=0.3)

In [10]:
watchman = GbmPredictWatchman(random_state=31)

In [11]:
for train, _, _ in tqdm(ds.train_generator()):
    watchman.prefit(train)
for train, _, _ in tqdm(ds.train_generator()):
    watchman.partial_fit(train)

0it [00:00, ?it/s]

0it [00:00, ?it/s]

In [ ]:
# watchman.limits

In [ ]:
# main_columns = ['rt_temp_gc', 'ht_temp_gc', 'rt_level_m', 'ht_level_m']

In [12]:
valid_gen = ds.valid_generator()
examine_list = pd.DataFrame(columns=['precision', 'recall', 'f1_score'], dtype='float')
for valid, faults, info in tqdm(valid_gen):
    detect = watchman.predict(valid)
#     detect = detect[main_columns]
    examine_list.loc[info] = time_span_metrics(faults, detect)
#     if max(faults) and max(detect):
#         plot_stacked(valid, faults=faults, detect=detect)
#         break

0it [00:00, ?it/s]

In [13]:
examine_list.mean()

precision    0.010890
recall       1.000000
f1_score     0.021537
dtype: float64

## TEP

In [19]:
ds = TepHarvardDataset()
ds.shake_not_stir(random_state=42, valid_test_ratio=1.0, balanced_test=True)

In [20]:
watchman = GbmPredictWatchman(random_state=31)

In [21]:
for train, _, _ in tqdm(ds.train_generator()):
    watchman.prefit(train)
for train, _, _ in tqdm(ds.train_generator()):
    watchman.partial_fit(train)

0it [00:00, ?it/s]

0it [00:00, ?it/s]

KeyboardInterrupt: 

In [ ]:
# watchman.limits

In [22]:
valid_gen = ds.valid_generator()
examine_list = pd.DataFrame(columns=['precision', 'recall', 'f1_score'], dtype='float')
for valid, faults, info in tqdm(valid_gen):
    detect = watchman.predict(valid)
    examine_list.loc[info] = time_span_metrics(faults, detect)

0it [00:00, ?it/s]

KeyboardInterrupt: 

In [23]:
examine_list.mean()

precision    0.960000
recall       1.000000
f1_score     0.979592
dtype: float64

## SWaT

In [24]:
ds = SwatItrustDataset()
ds.shake_not_stir(random_state=42, valid_test_ratio=0.5)

In [27]:
watchman = GbmPredictWatchman(random_state=31, use_log_state=False)

In [28]:
for train, _, _ in tqdm(ds.train_generator()):
    watchman.prefit(train)
for train, _, _ in tqdm(ds.train_generator()):
    watchman.partial_fit(train)

0it [00:00, ?it/s]

0it [00:00, ?it/s]

In [29]:
# watchman.limits

In [30]:
valid_gen = ds.valid_generator()
examine_list = pd.DataFrame(columns=['precision', 'recall', 'f1_score'], dtype='float')
for valid, faults, info in tqdm(valid_gen):
    detect = watchman.predict(valid)
    examine_list.loc[info] = time_span_metrics(faults, detect)

0it [00:00, ?it/s]

In [31]:
examine_list.mean()

precision    0.051724
recall       1.000000
f1_score     0.098361
dtype: float64